<a href="https://colab.research.google.com/github/hajaf/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [89.9 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ge

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-12 01:31:48--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.24MB/s    in 0.2s    

2022-08-12 01:31:48 (6.24 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_PC_v1_00.tsv.gz"), sep="\t", header=True)

In [5]:
# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [7]:
# Create the cstomers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   21374977|             1|
|   16281356|             1|
|   22073787|             1|
|   10245571|             7|
|   37757447|            16|
|   21977184|             2|
|     112911|             1|
|   15259255|             1|
|   27642109|             1|
|   41826621|             1|
|   38289873|            13|
|   10494993|             1|
|   27435626|             6|
|   41534903|             1|
|    1975358|             2|
|   22574219|             1|
|   22279960|             1|
|   47399638|             5|
|   22848554|             1|
|   51068983|            12|
+-----------+--------------+
only showing top 20 rows



In [8]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()
products_df.show



<bound method DataFrame.show of DataFrame[product_id: string, product_title: string]>

In [9]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent",to_date("review_date", 'yyyy-MM-dd').alias("review_date")])


In [10]:
review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('review_date', 'date')]

In [11]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3ARRMDEGED8RD|          5|            0|          0|   N|                Y|
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
| RUXJRZCT6953M|          1|            2|          2|   N|                N|
| R7EO0UO6BPB71|          1|            0|          0|   N|                Y|
|R39NJY2YJ1JFSV|          5|            0|          0|   N|                Y|
|R31SR7REWNX7CF|          5|            0|          0|   N|                Y|
| RVBP8I1R0CTZ8|          3|            1|          2|   N|                Y|
|R1QF6RS1PDLU18|          4|            1|          1|   N|                Y|
|R23AICGEDAJQL1|          1|            0|          0|   N|                Y|
|R2EY3N4K9W19UP|          5|            3|          4|   N|     

In [12]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cvygh0irovkf.us-east-1.rds.amazonaws.com:5432/"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [13]:
# Write review_id_df to table in RDS
#review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [14]:
# Write products_df to table in RDS
# about 3 min
#products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [15]:
# Write customers_df to table in RDS
# 5 min 14 s
#customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [16]:
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3ARRMDEGED8RD|          5|            0|          0|   N|                Y|
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
| RUXJRZCT6953M|          1|            2|          2|   N|                N|
| R7EO0UO6BPB71|          1|            0|          0|   N|                Y|
|R39NJY2YJ1JFSV|          5|            0|          0|   N|                Y|
|R31SR7REWNX7CF|          5|            0|          0|   N|                Y|
| RVBP8I1R0CTZ8|          3|            1|          2|   N|                Y|
|R1QF6RS1PDLU18|          4|            1|          1|   N|                Y|
|R23AICGEDAJQL1|          1|            0|          0|   N|                Y|
|R2EY3N4K9W19UP|          5|            3|          4|   N|     

In [17]:

#total_votes count is equal to or greater than 20 
from pyspark.sql.functions import col
filter1_df =vine_df.filter(col("total_votes") >= 20)
filter1_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R35OW0SJPJWAHO|          1|           12|        162|   N|                Y|
| RAP2ETU511BOI|          1|            5|         99|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R1RMOCWMV8F4VH|          1|            5|         21|   N|                Y|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|
|R38OLNY5GR1PW6|          5|           44|         44|   N|     

In [18]:
# create a new DataFrame retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
#dataframe.withColumn('half_price',dataframe['price']/2).show()
filter2_df=filter1_df.filter(col("helpful_votes")/col("total_votes")>=0.5)
filter2_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|
|R38OLNY5GR1PW6|          5|           44|         44|   N|                Y|
|R2GK7RJNIAREIT|          1|           30|         32|   N|                Y|
|R2OWRP6FXXECMK|          1|           42|         44|   N|                Y|
| ROR2FE6Y8EZYG|          5|           19|         22|   N|     

In [19]:
filter3_df=filter2_df.filter(col("vine")=="Y")
filter3_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3J2DPYUH6453D|          5|          136|        153|   Y|                N|
|R25QTFLERWK6SJ|          3|          852|        956|   Y|                N|
|R1BG71K9B61LI1|          4|           39|         39|   Y|                N|
| R8WGX426U56E0|          5|           19|         21|   Y|                N|
| R614TCIDD36LE|          4|          131|        147|   Y|                N|
|R28X5K1OZBQSBK|          5|           25|         28|   Y|                N|
|R2CUDWKAUWGHIT|          5|           19|         24|   Y|                N|
| R886ADSI1HBU5|          5|           88|         97|   Y|                N|
|R3TTJTAND1CCSV|          5|           21|         23|   Y|                N|
|R1W39WXYWNFJBG|          5|           32|         35|   Y|     

In [43]:
vine_review_df=filter3_df.agg({'review_id': 'count'})
vine_review_df.show()

+----------------+
|count(review_id)|
+----------------+
|            1775|
+----------------+



In [20]:
filter4_df=filter2_df.filter(col("vine")=="N")
filter4_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|
|R38OLNY5GR1PW6|          5|           44|         44|   N|                Y|
|R2GK7RJNIAREIT|          1|           30|         32|   N|                Y|
|R2OWRP6FXXECMK|          1|           42|         44|   N|                Y|
| ROR2FE6Y8EZYG|          5|           19|         22|   N|     

In [21]:
filter5_df=filter4_df.agg({'review_id': 'count'})
filter5_df.show()

+----------------+
|count(review_id)|
+----------------+
|           77385|
+----------------+



In [44]:
#the number of 5-star vine reviews
filter6_df=filter3_df.filter(col("star_rating") ==5)
filter6_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3J2DPYUH6453D|          5|          136|        153|   Y|                N|
| R8WGX426U56E0|          5|           19|         21|   Y|                N|
|R28X5K1OZBQSBK|          5|           25|         28|   Y|                N|
|R2CUDWKAUWGHIT|          5|           19|         24|   Y|                N|
| R886ADSI1HBU5|          5|           88|         97|   Y|                N|
|R3TTJTAND1CCSV|          5|           21|         23|   Y|                N|
|R1W39WXYWNFJBG|          5|           32|         35|   Y|                N|
|R2UGZFCPGAI6OP|          5|           19|         21|   Y|                N|
|R1FI3ZKOA0RMR9|          5|          145|        164|   Y|                N|
|R3GNG9SISW41OZ|          5|          143|        153|   Y|     

In [45]:
#the number of 5-star vine reviews
filter7_df=filter6_df.agg({'star_rating': 'count'})
filter7_df.show()

+------------------+
|count(star_rating)|
+------------------+
|               783|
+------------------+



In [47]:
#the number of 5-star non-vine reviews
five_star_nonvine_df=filter4_df.filter(col("star_rating") ==5)
five_star_nonvine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R38OLNY5GR1PW6|          5|           44|         44|   N|                Y|
| ROR2FE6Y8EZYG|          5|           19|         22|   N|                Y|
| RIJYX8PG45XOB|          5|           51|         52|   N|                Y|
| RT880ROJ1I4Z5|          5|           22|         31|   N|                N|
|R3H11HIP4SCL9V|          5|           31|         33|   N|                Y|
|R2UAVRDG3AUF3K|          5|           14|         21|   N|                N|
| RG5MCPWSDW1G5|          5|           57|         65|   N|                Y|
|R3G2VF93TYJIYU|          5|           44|         48|   N|                N|
|R3PLNRCAIHS539|          5|           64|         66|   N|     

In [50]:
filters_df=five_star_nonvine_df.agg({'star_rating': 'count'})
filters_df.show()

+------------------+
|count(star_rating)|
+------------------+
|             35949|
+------------------+



In [55]:
# the percentage 5-star reviews are calculated for all Vine 
783/1775 *100

44.11267605633803

In [56]:

 # the percentage 5-star reviews are calculated for all  non-Vine reviews 
35949/77385 *100

46.454739290560184

In [24]:
#the percentage of 5-star reviews for the two types of review (paid vs unpaid).
#paid
filter8_df=vine_df.filter(col("verified_purchase")=="Y")
filter8_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3ARRMDEGED8RD|          5|            0|          0|   N|                Y|
| R7EO0UO6BPB71|          1|            0|          0|   N|                Y|
|R39NJY2YJ1JFSV|          5|            0|          0|   N|                Y|
|R31SR7REWNX7CF|          5|            0|          0|   N|                Y|
| RVBP8I1R0CTZ8|          3|            1|          2|   N|                Y|
|R1QF6RS1PDLU18|          4|            1|          1|   N|                Y|
|R23AICGEDAJQL1|          1|            0|          0|   N|                Y|
|R2EY3N4K9W19UP|          5|            3|          4|   N|                Y|
| RC9AW4HKJ016M|          1|            0|          0|   N|                Y|
|R2G5FPA4OX37GV|          5|            1|          1|   N|     

In [25]:
filter9_df=filter8_df.filter(col("star_rating") ==5)
filter9_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3ARRMDEGED8RD|          5|            0|          0|   N|                Y|
|R39NJY2YJ1JFSV|          5|            0|          0|   N|                Y|
|R31SR7REWNX7CF|          5|            0|          0|   N|                Y|
|R2EY3N4K9W19UP|          5|            3|          4|   N|                Y|
|R2G5FPA4OX37GV|          5|            1|          1|   N|                Y|
|R1IKTSEVXSIMOD|          5|            0|          0|   N|                Y|
| RS9H1N9I3Z1IA|          5|            0|          0|   N|                Y|
|R21PTQNLGCBN0I|          5|            0|          0|   N|                Y|
| RDVE5MHPKF0R8|          5|            0|          0|   N|                Y|
|  R65UQSAULGB5|          5|            0|          0|   N|     

In [26]:

filter10_df=filter9_df.agg({'star_rating': 'count'})
filter10_df.show()

+------------------+
|count(star_rating)|
+------------------+
|           3681067|
+------------------+



In [38]:
filter12_df=filter8_df.agg({'star_rating': 'count'})
filter12_df.show()


+------------------+
|count(star_rating)|
+------------------+
|           6048849|
+------------------+



In [39]:
#The percentage of paid 5 start reveiews
3681067/ 6048849*100

60.85566030826691

In [30]:
#unpaid
filter13_df=vine_df.filter(col("verified_purchase")=="N")
filter13_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
| RUXJRZCT6953M|          1|            2|          2|   N|                N|
|R2ALWJE9N6ZBXD|          1|            0|          0|   N|                N|
|R2RATALMIFYU7H|          4|            0|          0|   N|                N|
| R9GYKTMBT2LLT|          5|            0|          0|   N|                N|
|R2QEQK2UHCZSFZ|          4|            2|          2|   Y|                N|
|R37IANVKK3ZZUU|          3|            0|          0|   N|                N|
|R1IRZI951LUFP9|          5|            1|          2|   N|                N|
| RKP98DQX0VTFZ|          5|            1|          3|   N|                N|
|R1R9KCG476TLMT|          5|            0|          0|   N|     

In [31]:
filter14_df=filter13_df.filter(col("star_rating") ==5)
filter14_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
| R9GYKTMBT2LLT|          5|            0|          0|   N|                N|
|R1IRZI951LUFP9|          5|            1|          2|   N|                N|
| RKP98DQX0VTFZ|          5|            1|          3|   N|                N|
|R1R9KCG476TLMT|          5|            0|          0|   N|                N|
|R1RBOXG2Q04EAB|          5|            0|          0|   N|                N|
|R140OYEPD13BBW|          5|            0|          0|   N|                N|
|R16TSWE5MMLG2X|          5|            0|          0|   N|                N|
|R1CWZZX73S8OSJ|          5|            0|          2|   N|                N|
|R11VKF0G56SID6|          5|            0|          0|   N|     

In [41]:
filter15_df=filter14_df.agg({'star_rating': 'count'})
filter15_df.show()


+------------------+
|count(star_rating)|
+------------------+
|            425798|
+------------------+



In [40]:
filter15_df=filter13_df.agg({'star_rating': 'count'})
filter15_df.show()


+------------------+
|count(star_rating)|
+------------------+
|            859702|
+------------------+



In [42]:
#The percentage of unpaid 5 start reveiews
425798/859702 *100

49.52855756994865